In [43]:
import pandas as pd
import requests 
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
from openai import OpenAI
import json
from decouple import config
import tiktoken
# from decouple import config
encoding = tiktoken.encoding_for_model("gpt-4o")
es = Elasticsearch("http://localhost:9200")
# client = OpenAI(
#     base_url='http://localhost:11434/v1/',
#     api_key='ollama',
# )
# print(os.environ.get("api_key"))
client = OpenAI(
    api_key=config("api_key"),
    base_url="https://api.aimlapi.com",
)

In [36]:
def get_documents():
    docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()
    # print(documents_raw)
    documents = []
    
    for course in documents_raw:
        course_name = course['course']
        # print(course_name)
        for doc in course['documents']:
            doc['course'] = course_name
            documents.append(doc)
    return documents

In [37]:
def setup_index(index_name='course', documents=None):
    index_settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                "text": {"type": "text"},
                "section": {"type": "text"},
                "question": {"type": "text"},
                "course": {"type": "keyword"} 
            }
        }
    }
    if not es.indices.exists(index=index_name):
        response = es.indices.create(index=index_name, body=index_settings)
    
    for doc in tqdm(documents):
        es.index(index=index_name, document=doc)
    return index_name

In [38]:
def retrieval(query, index_name='course', max_results=3):
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es.search(index=index_name, body=search_query)
    top_n_matched_questions = [hit['_source'] for hit in response['hits']['hits']]
    return top_n_matched_questions

In [44]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()


def build_context(documents):
    context_result = ""
    
    for doc in documents:
        doc_str = context_template.format(**doc)
        context_result += ("\n\n" + doc_str)
    
    return context_result.strip()


def build_prompt(user_question, documents):
    context = build_context(documents)
    prompt = prompt_template.format(
        question=user_question,
        context=context
    )
    return prompt

def ask_openai(prompt, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model='mistralai/Mistral-7B-Instruct-v0.2',
        messages=[{"role": "user", "content": prompt}]
    )
    answer = response.choices[0].message.content
    return answer

def qa_bot(user_question):
    context_docs = retrieval(user_question)
    # print(json.dumps(context_docs, indent=4))
    prompt = build_prompt(user_question, context_docs)
    print("len of prompt",len(prompt))
    print("tokens of prompt", len(encoding.encode(prompt)))
    answer = ask_openai(prompt, "phi3")
    print(answer)
    return answer

In [40]:
# we need to include the documents in the elastic_search, only after that we can run rest of the processes of prompting
def run():
    documents = get_documents()
    index_name = setup_index("courses", documents)

In [45]:
run()

100%|██████████| 948/948 [00:01<00:00, 801.43it/s]


In [46]:

qa_bot("How do I execute a command in a running docker container?")

len of prompt 1322
tokens of prompt 298
 To execute a command in a running Docker container, follow these steps:

1. Find the container ID using the command `docker ps`.
2. Execute the command in the container using the command `docker exec -it <container-id> <command>`. Replace `<container-id>` with the actual container ID and `<command>` with the command you want to execute. In this case, replace `<command>` with `bash` if you want to open a bash shell in the container.


' To execute a command in a running Docker container, follow these steps:\n\n1. Find the container ID using the command `docker ps`.\n2. Execute the command in the container using the command `docker exec -it <container-id> <command>`. Replace `<container-id>` with the actual container ID and `<command>` with the command you want to execute. In this case, replace `<command>` with `bash` if you want to open a bash shell in the container.'

In [33]:
!pip install tiktoken

     |████████████████████████████████| 907 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 278 kB 70.6 MB/s eta 0:00:01
You should consider upgrading via the '/Users/salahuddinpalagiri/Desktop/Repos/Mera/llm-zoomcamp/search-implementation/llm-zoom/bin/python3 -m pip install --upgrade pip' command.


In [34]:
import tiktoken

